<a href="https://colab.research.google.com/github/MarcioLuizBR/desafio_imersaoIA2025/blob/main/HELPTEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finalização da Imersão IA 3ª Edição da Alura com Google Gemini


### Projeto "HELPTEA" (Ajuda para o Transtornos do Espectro Autista)

Desafio de desenvolvimento de uma aplicação pratica dos conceitos apresentados na imersão Alura/Google Gemini

#### instalação dos pacotes essenciais

In [44]:
!pip -q install google-generativeai
!pip -q install google-adk

In [45]:
import os, textwrap, warnings
import requests # Para fazer requisições HTTP
from google.colab import userdata
from google import generativeai as genai
from datetime import date
from IPython.display import HTML, Markdown, display
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)


warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
MODEL_ID = "gemini-2.0-flash"

In [46]:
from google.generativeai import GenerativeModel
modelo = GenerativeModel("gemini-2.0-flash")

agente_assistenteSocial = modelo.start_chat(history=[])
agente_juridico = modelo.start_chat(history=[])
agente_educacional = modelo.start_chat(history=[])
agente_nutricionista = modelo.start_chat(history=[])
agente_psicologo = modelo.start_chat(history=[])
agente_redator = modelo.start_chat(history=[])


#### Função auxiliar para exibir texto formatado em Markdown no Colab

In [47]:
def to_markdown(text):
    """Exibe Markdown bonito no Colab."""
    text = text.replace('•', '  *')
    display(Markdown(textwrap.indent(text, "> ", lambda _: True)))

#### Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

In [48]:
def call_agent(agent: Agent, user_message: str, user_id="user1", session_id="s1"):
    """Roda o agente via Runner e devolve a resposta final (string)."""
    sess_service = InMemorySessionService()
    sess_service.create_session(app_name=agent.name, user_id=user_id, session_id=session_id)
    runner = Runner(agent=agent, app_name=agent.name, session_service=sess_service)
    content = types.Content(role="user", parts=[types.Part(text=user_message)])

    final_text = ""
    for ev in runner.run(user_id=user_id, session_id=session_id, new_message=content):
        if ev.is_final_response():
            for p in ev.content.parts:
                if p.text:
                    final_text += p.text + "\n"
    return final_text.strip()

#### Função que constrói TODOS os agentes

In [ ]:
# -------------------------------------------
# 🚀 DEFINIÇÃO DOS AGENTES (sem localizador)
# -------------------------------------------
def build_agents() -> dict:
    hoje = date.today().strftime("%d/%m/%Y")

    return {
        "assistente_social": Agent(
            name="assistente_social",
            model=MODEL_ID,
            description="Orientação prática sobre benefícios sociais, serviços públicos e ONGs.",
            instruction=f"""
                Você é Assistente Social especializado em TEA.
                • Fale de forma acolhedora e objetiva.
                • Traga exemplos de benefícios (BPC, passe livre, etc.).
                • Data de referência: {hoje}.
            """,
            tools=[google_search]
        ),

        "juridico": Agent(
            name="juridico",
            model=MODEL_ID,
            description="Direitos legais da pessoa autista.",
            instruction=f"""
                Você é advogado especializado em TEA.
                • Cite leis (12.764/2012, LBI 13.146/2015) quando útil.
                • Seja prático e sem jargões excessivos.
                • Data de referência: {hoje}.
            """,
            tools=[google_search]
        ),

        "educacional": Agent(
            name="educacional",
            model=MODEL_ID,
            description="Estratégias pedagógicas inclusivas.",
            instruction="""
                Você é pedagogo especialista em educação inclusiva para TEA.
                • Sugira adaptações simples, rotinas visuais, TEACCH/PECS.
            """,
            tools=[google_search]
        ),

        "nutricionista": Agent(
            name="nutricionista",
            model=MODEL_ID,
            description="Recomendações sobre seletividade alimentar e hábitos saudáveis.",
            instruction="""
                Você é nutricionista especializado em TEA.
                • Dê dicas práticas para ampliar repertório alimentar.
                • Oriente sempre consultar profissional presencial.
            """,
            tools=[google_search]
        ),

        "psicologo": Agent(
            name="psicologo",
            model=MODEL_ID,
            description="Apoio emocional e manejo comportamental.",
            instruction="""
                Você é psicólogo infantil com foco em TEA.
                • Traga estratégias breves de regulação emocional e reforço positivo.
            """
        ),

        "redator_final": Agent(
            name="redator_final",
            model=MODEL_ID,
            description="Resume de forma clara, empática e concisa.",
            instruction="""
                Recebe as respostas dos especialistas e:
                • Cria um texto final coeso de no máx. 4 parágrafos curtos.
                • Lista (em tópicos) 3 referências ou links úteis ao final.
            """
        ),
    }

agentes = build_agents()   # cria todos



In [ ]:
# -------------------------------------------
# 🛠️ FUNÇÃO PRINCIPAL
# -------------------------------------------
def gerar_resposta(topic: str) -> str:
    """Dispara os 5 especialistas e depois o redator."""
    especialistas = ["assistente_social", "juridico",
                     "educacional", "nutricionista", "psicologo"]

    respostas = []
    for chave in especialistas:
        prompt = f"Tema: {topic}\nElabore orientação dentro da sua especialidade."
        respostas.append(f"{chave}:\n" + call_agent(agentes[chave], prompt))

    # Junta tudo para o redator
    material = "\n\n".join(respostas)
    resumo = call_agent(agentes["redator_final"], material)
    return resumo

In [ ]:
# -------------------------------------------
# 💬 FLUXO DE INTERAÇÃO
# -------------------------------------------
tema_usuario = input("Digite o tema que deseja saber (ex.: alimentação seletiva): ").strip()

if tema_usuario:
    resultado = gerar_resposta(tema_usuario)
    to_markdown(resultado)
else:
    print("⚠️  Você precisa digitar um tema.")